## LLM-as-a-Judge Evaluation
Evaluate the end-to-end performance using LLM-as-a-Judge provided by Ragas.

#### 1. Context Precision (without reference)

**Context Precision** is a metric that evaluates the retriever's ability to rank relevant chunks higher than irrelevant ones for a given query in the retrieved context. Specifically, it assesses the degree to which relevant chunks in the retrieved context are placed at the top of the ranking.

It is calculated as the mean of the `precision@k` for each chunk in the context. `Precision@k` is the ratio of the number of relevant chunks at rank k to the total number of chunks at rank k.

In [1]:
pip install ragas

  Using cached ragas-0.4.1-py3-none-any.whl.metadata (22 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached instructor-1.13.0-py3-none-any.whl.metadata (11 kB)
  Using cached scikit_network-0.33.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.5 kB)
  Using cached jiter-0.11.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pre_commit-4.5.0-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached ty-0.0.1a33-py3-none-macosx_11_0_arm64.whl.metadata (4.5 kB)
  Using cached cfgv-3.5.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached identify-2.6.15-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached nodeenv-1.9.1-py2.py3-none-any.whl.metadata (21 kB)
  Using cached virtualenv-20.35.4-py3-none-any.whl.metadata (4.6 kB)
  Using cached distlib-0.4.0-py2.py3-none-any.whl.metadata (5.2 kB)
Using cached ragas-0.4.1-py3-none-any.whl (419 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached instructor-1.13.0-py3-none-any.whl

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

from datasets import Dataset
from ragas import evaluate, SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference

from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI

/opt/anaconda3/envs/voice-rag-py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
evaluator_llm = ChatOpenAI(
    model=os.getenv("RAGAS_EVAL_MODEL", "gpt-4o"),
    temperature=0,
)

wrapped_evaluator_llm = LangchainLLMWrapper(evaluator_llm)

/var/folders/zr/hq5f8g3s0jq6yt_pl2ppbgmc0000gn/T/ipykernel_32904/289931900.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  wrapped_evaluator_llm = LangchainLLMWrapper(evaluator_llm)


In [ ]:
from typing import List, Dict, Any
from graph.graph import app

def run_agent_and_collect(
    queries: List[str],
) -> List[Dict[str, Any]]:
    runs: List[Dict[str, Any]] = []

    for q in queries:
        result = app.invoke({"input": q})

        user_input = q

        final_response = result.get("response", "")

        # 3) retrieved_contexts
        #    ⚠️ 여기 부분은 너의 RAG 구현에 맞게 수정해야 함
        #    예: retrieval_tool 에서 products / web_results 원본 컨텍스트를 state에 같이 넣어두고
        #        거기서 텍스트만 뽑아서 리스트로 만든다고 가정
        #
        #    아래는 placeholder 예시:
        retrieved_contexts = result.get("documents", [])

        if not isinstance(retrieved_contexts, list):
            retrieved_contexts = [str(retrieved_contexts)]

        runs.append(
            {
                "user_input": user_input,
                "response": final_response,
                "retrieved_contexts": retrieved_contexts,
            }
        )

    return runs


test_queries = [
    "I want to find a stuffed animal for kids less than $30"
]

runs = run_agent_and_collect(test_queries)

len(runs), runs[0]


ROUTER NODE Started
router response:  * Task:  
  Find a stuffed animal for kids.

* Constraints:  
  - Budget: Less than $30  
  - Material: Not specified  
  - Brand: Not specified  

* Safety Flags: No
Planner NODE Started
planner response:  * Data Source: private
* Fields to Retrieve: product descriptions, prices, ratings
* Comparison Criteria: price, rating
Retriever NODE Started
RETRIEVAL TOOL Invoked with query: Retrieve products based on this plan:
* Data Source: private
* Fields to Retrieve: product descriptions, prices, ratings
* Comparison Criteria: price, rating
[MCP] Calling http://0.0.0.0:8001/tools/rag.search with args={'query': 'Retrieve products based on this plan:\n* Data Source: private\n* Fields to Retrieve: product descriptions, prices, ratings\n* Comparison Criteria: price, rating', 'top_k': 5, 'max_price': None, 'min_rating': None, 'brand': None}
[MCP] tool=rag.search response type: <class 'dict'>
[MCP] Calling http://0.0.0.0:8001/tools/web.search with args={'que

(1,
 {'user_input': 'I want to find a stuffed animal for kids less than $30',
  'response': '* Final Answer:  \nBased on the retrieved knowledge, a suitable option for a stuffed animal for kids under $30 is the "Melissa & Doug Annie Doll & Feeding Set Bundle," priced at $24.99 with a rating of 4.4 [3]. This product fits within your budget and is from a reputable brand known for children\'s toys. There are no safety concerns noted in the retrieved data.',
  'retrieved_contexts': []})

In [11]:
from ragas import SingleTurnSample

def to_single_turn_samples(runs: List[Dict[str, Any]]) -> List[SingleTurnSample]:
    samples: List[SingleTurnSample] = []

    for r in runs:
        sample = SingleTurnSample(
            user_input=r["user_input"],
            response=r["response"],
            retrieved_contexts=r["retrieved_contexts"],
        )
        samples.append(sample)

    return samples

samples = to_single_turn_samples(runs)
len(samples), samples[0]


(1,
 SingleTurnSample(user_input='I want to find a stuffed animal for kids less than $30', retrieved_contexts=[], reference_contexts=None, retrieved_context_ids=None, reference_context_ids=None, response='* Final Answer:  \nBased on the retrieved knowledge, a suitable option for a stuffed animal for kids under $30 is the "Melissa & Doug Annie Doll & Feeding Set Bundle," priced at $24.99 with a rating of 4.4 [3]. This product fits within your budget and is from a reputable brand known for children\'s toys. There are no safety concerns noted in the retrieved data.', multi_responses=None, reference=None, rubrics=None, persona_name=None, query_style=None, query_length=None))

In [12]:
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference

context_precision_metric = LLMContextPrecisionWithoutReference(
    llm=wrapped_evaluator_llm
)

async def score_one_sample(sample: SingleTurnSample) -> float:
    score = await context_precision_metric.single_turn_ascore(sample)
    return score

one_score = await context_precision_metric.single_turn_ascore(samples[0])
print("First sample LLM Context Precision (no reference):", one_score)

First sample LLM Context Precision (no reference): 0.0
